In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("../data/raw/teachers_db_practice.csv")

# Take a quick look
print(df.shape)
df.head()


In [ ]:
df.columns
print(df.loc[0, "full_info"])

In [ ]:
import re

# Remove HTML tags and extra whitespace
df["clean_text"] = (
    df["full_info"]
    .fillna("")
    .apply(lambda x: re.sub(r"<.*?>", " ", x))
    .apply(lambda x: re.sub(r"\s+", " ", x).strip())
)

df["clean_text"].head(3).values

In [ ]:
# ---- Extract sections from the original HTML-ish text ----
import re

SECTION_HEADERS = {
    "corp": r"(?:<h4>\s*CORPORATE EXPERIENCE\s*</h4>|CORPORATE EXPERIENCE)",
    "acadexp": r"(?:<h4>\s*ACADEMIC EXPERIENCE\s*</h4>|ACADEMIC EXPERIENCE)",
    "acadbg": r"(?:<h4>\s*ACADEMIC BACKGROUND\s*</h4>|ACADEMIC BACKGROUND)",
}

def strip_html(s: str) -> str:
    if not isinstance(s, str):
        return ""
    # fix common &amp; noise then strip tags
    s = re.sub(r"&\s*amp;?", "&", s, flags=re.I)
    s = re.sub(r"<.*?>", " ", s)              # remove tags
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_sections(raw: str):
    text = raw if isinstance(raw, str) else ""
    # Normalize case for header matching but keep original for content slicing
    low = text.lower()

    def find_idx(pattern):
        m = re.search(pattern, text, flags=re.I)
        return m.start() if m else None

    i_corp = find_idx(SECTION_HEADERS["corp"])
    i_acadexp = find_idx(SECTION_HEADERS["acadexp"])
    i_acadbg = find_idx(SECTION_HEADERS["acadbg"])

    # helper to slice safely
    def slice_between(start, end):
        if start is None:
            return ""
        end = len(text) if end is None else end
        return strip_html(text[start:end])

    # determine boundaries
    idxs = sorted([(k, v) for k, v in [("corp", i_corp), ("acadexp", i_acadexp), ("acadbg", i_acadbg)] if v is not None],
                  key=lambda x: x[1])
    corp_txt = acadexp_txt = acadbg_txt = ""
    if idxs:
        # walk through in order and slice to next header
        for j, (label, start) in enumerate(idxs):
            end = idxs[j+1][1] if j+1 < len(idxs) else None
            chunk = slice_between(start, end)
            if label == "corp": corp_txt = chunk
            elif label == "acadexp": acadexp_txt = chunk
            elif label == "acadbg": acadbg_txt = chunk
    else:
        # fallback: no headers -> treat all as one generic blob (will still be used if needed)
        corp_txt = strip_html(text)

    return pd.Series({
        "corp_text": corp_txt,
        "acadexp_text": acadexp_txt,
        "acadbg_text": acadbg_txt
    })

sec_df = df["full_info"].apply(extract_sections)
df = pd.concat([df, sec_df], axis=1)


In [ ]:
from transformers import pipeline

# Load a pre-trained NER model
ner = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

# Test on small sample of dataset
sample_text = df.loc[1, "clean_text"]
ner_results = ner(sample_text[:1000])  # limit to 1000 chars just to test
ner_results[:10]



In [ ]:
from tqdm import tqdm
tqdm.pandas()

df["entities"] = df["clean_text"].progress_apply(lambda x: ner(x[:2000]))


In [ ]:
# ---- Run NER per section (safer for dict building) ----
from tqdm import tqdm
tqdm.pandas()

def run_ner_safe(s, limit=2000):
    s = s if isinstance(s, str) else ""
    return ner(s[:limit]) if s else []

df["entities_corp"]    = df["corp_text"].progress_apply(run_ner_safe)
df["entities_acadexp"] = df["acadexp_text"].progress_apply(run_ner_safe)
df["entities_acadbg"]  = df["acadbg_text"].progress_apply(run_ner_safe)


In [ ]:
from rapidfuzz import process, fuzz

def normalize_entities(entities):
    cleaned = []
    for ent in entities:
        word = ent["word"].strip()
        # Remove stray punctuation and unify case
        word = word.replace(".", "").replace(",", "").title()
        cleaned.append((ent["entity_group"], word))
    return cleaned

df["normalized_entities"] = df["entities"].apply(normalize_entities)
df["normalized_entities"].head(2).values



In [ ]:
import re

LOCATION_MAP = {
    "us": "United States",
    "usa": "United States",
    "u.s.": "United States",
    "u.k.": "United Kingdom",
    "uk": "United Kingdom",
    "spain": "Spain",
    "madrid": "Madrid",
    "mexico": "Mexico",
    "mexico city": "Mexico City",
    "london": "London",
    "paris": "Paris",
    "france": "France",
    "germany": "Germany",
    "italy": "Italy",
    "portugal": "Portugal",
    "barcelona": "Barcelona",
    "new york": "New York",
}

ORG_FIXES = {
    # IE ecosystem
    "ie": "IE",
    "ie university": "IE",
    "ie business school": "IE",
    "ie law school": "IE",
    "instituto de empresa": "IE",
    "ie school of global and public affairs": "IE",
    # Spanish universities
    "universidad autonoma de madrid": "Universidad Autónoma de Madrid",
    "uam": "Universidad Autónoma de Madrid",
    "universidad complutense de madrid": "Universidad Complutense de Madrid",
    "universidad carlos iii de madrid": "Universidad Carlos III de Madrid",
    "universidad politecnica de madrid": "Universidad Politécnica de Madrid",
    "universidad de navarra": "Universidad de Navarra",
    "universidad pontificia comillas": "Universidad Pontificia Comillas",
    "comillas university": "Universidad Pontificia Comillas",
    "icade": "Universidad Pontificia Comillas",
    "iese business school": "IESE Business School",
    # companies / misc
    "a & am": "A&M Studio",
    "a&m": "A&M Studio",
    "am studio": "A&M Studio",
}

GENERIC_ORG_WORDS = {
    "academic", "academic exp", "academ", "experience", "exp",
    "university", "universidad", "engineering", "design",
    "academy", "school", "faculty", "department", "college",
    "education", "institute", "business", "finance", "management",
    "administration", "economics", "marketing", "law", "science",
    "technology", "research", "professor", "lecturer"
}

def normalize_entities(entities):
    cleaned = []
    for ent in entities:
        text = ent["word"].lower().strip()
        text = re.sub(r"[^a-z0-9&.\sáéíóúüñ]", "", text)
        text = re.sub(r"\s+", " ", text).strip()

        if len(text) < 3:
            continue

        if ent["entity_group"] == "LOC":
            text = LOCATION_MAP.get(text, text.title())

        elif ent["entity_group"] == "ORG":
            text = ORG_FIXES.get(text, text.title())

            # drop headings and generic garbage
            if (
                text.lower() in GENERIC_ORG_WORDS
                or len(text.split()) == 1 and text.lower() not in [v.lower() for v in ORG_FIXES.values()]
            ):
                continue

        else:
            text = text.title()

        cleaned.append((ent["entity_group"], text))
    return cleaned

df["normalized_entities"] = df["entities"].apply(normalize_entities)


In [ ]:
# ---- Normalize per section ----
df["norm_corp"]    = df["entities_corp"].apply(normalize_entities)
df["norm_acadexp"] = df["entities_acadexp"].apply(normalize_entities)
df["norm_acadbg"]  = df["entities_acadbg"].apply(normalize_entities)

# Optional: fuzzy consolidate inside each section
df = consolidate_similar_entities(df, "ORG", threshold=90)  # keeps df["normalized_entities"], but we’ll use the sectioned cols
# (If you want the consolidate step section-specific, you can skip it here; your maps already did most of the work.)


In [ ]:
# -------- SECTION-AWARE EXTRACTION (place directly under the last normalized_entities assignment) --------
import re
from html import unescape

def _extract_section(html_text, header):
    """Grab text under <h4>HEADER</h4> until the next <h4> or end."""
    if not isinstance(html_text, str):
        return ""
    s = unescape(html_text)  # fixes '&amp;' -> '&' so we don't get '& Am'
    # capture everything after this header up to next <h4> or end
    m = re.search(rf"<h4>\s*{header}\s*</h4>(.*?)(?=<h4>|$)", s, flags=re.I|re.S)
    if not m:
        return ""
    # strip tags but keep text
    body = re.sub(r"<.*?>", " ", m.group(1))
    body = re.sub(r"\s+", " ", body).strip()
    return body

# 1) Parse sections from the original HTML (not the stripped text)
df["section_corporate"] = df["full_info"].apply(lambda x: _extract_section(x, "CORPORATE EXPERIENCE"))
df["section_acad_exp"]  = df["full_info"].apply(lambda x: _extract_section(x, "ACADEMIC EXPERIENCE"))
df["section_acad_bg"]   = df["full_info"].apply(lambda x: _extract_section(x, "ACADEMIC BACKGROUND"))

# 2) Run NER per section (short-circuit empty strings; limit length to be laptop-friendly)
def _safe_ner(txt):
    if not txt:
        return []
    return ner(txt[:2000])  # reuse your existing HF pipeline

df["corp_entities"] = df["section_corporate"].apply(_safe_ner)
df["acadexp_entities"] = df["section_acad_exp"].apply(_safe_ner)
df["acadbg_entities"] = df["section_acad_bg"].apply(_safe_ner)

# 3) Reuse your normalization on each section
df["corp_norm"]    = df["corp_entities"].apply(normalize_entities)
df["acadexp_norm"] = df["acadexp_entities"].apply(normalize_entities)
df["acadbg_norm"]  = df["acadbg_entities"].apply(normalize_entities)

# 4) Build the assignment-style dictionary PER PROFESSOR using sectioned entities
def _pick(ents, label): 
    return [e[1] for e in ents if e[0] == label]

def build_professor_dict_from_sections(row):
    corp, aexp, abg = row["corp_norm"], row["acadexp_norm"], row["acadbg_norm"]
    return {
        "Corporate Experience - Organization": _pick(corp, "ORG"),
        "Corporate Experience - Location":     _pick(corp, "LOC"),
        "Academic Background - Organization":  _pick(abg,  "ORG"),
        "Academic Background - Education":     [],   # optional: fill later with regex/keywords
        "Academic Experience - Courses":       [],   # optional
        "Academic Experience - Subjects":      []    # optional
    }

df["professor_dict"] = df.apply(build_professor_dict_from_sections, axis=1)


In [ ]:

# --- Auto-alias clustering for ORG/LOC without hardcoding ---

import unicodedata
from collections import Counter, defaultdict
from rapidfuzz import fuzz, process

def simplify_for_match(s: str) -> str:
    # lowercase, strip accents, remove stop words like university/universidad/etc., collapse spaces
    s0 = s.lower().strip()
    s0 = "".join(c for c in unicodedata.normalize("NFKD", s0) if not unicodedata.combining(c))
    s0 = s0.replace("&", "and")
    # drop very generic words in many languages
    drop = {"university","universidad","universite","università","universita","universidade",
            "school","college","institute","instituto","universidad","dept","department"}
    tokens = [t for t in re.sub(r"[^a-z0-9\s]", " ", s0).split() if t not in drop]
    return " ".join(tokens)

# collect all ORG/LOC surface forms
all_orgs = []
all_locs = []
for row in df["normalized_entities"]:
    for typ, val in row:
        if typ == "ORG":
            all_orgs.append(val)
        elif typ == "LOC":
            all_locs.append(val)

def build_alias_map(names, sim_threshold=92):
    names = list(set(names))
    # group by simplified key first (fast win)
    buckets = defaultdict(list)
    for n in names:
        buckets[simplify_for_match(n)].append(n)

    alias_map = {}

    # within each bucket, pick the most common as canonical; also fuzzy-merge near buckets
    # pick canonical = most frequent original form
    freq = Counter(names)

    # first pass: intra-bucket
    for _, variants in buckets.items():
        canonical = max(variants, key=lambda x: freq[x])
        for v in variants:
            alias_map[v] = canonical

    # second pass: inter-bucket fuzzy consolidation of canonicals
    canonicals = list(set(alias_map[v] for v in alias_map))
    for c in canonicals:
        # find nearest other canonicals by token_sort_ratio
        matches = process.extract(c, canonicals, scorer=fuzz.token_sort_ratio, limit=5)
        for other, score, _ in matches:
            if other != c and score >= sim_threshold:
                # unify to the most frequent of the two
                winner = c if freq[c] >= freq[other] else other
                loser  = other if winner == c else c
                # redirect all aliases pointing to loser → winner
                for k, v in list(alias_map.items()):
                    if v == loser:
                        alias_map[k] = winner
                # also update the canonical list
                canonicals = [winner if x == loser else x for x in canonicals]

    return alias_map

ORG_ALIAS = build_alias_map(all_orgs, sim_threshold=92)
LOC_ALIAS = build_alias_map(all_locs, sim_threshold=95)

def apply_aliases(entities):
    out = []
    for typ, val in entities:
        if typ == "ORG":
            out.append((typ, ORG_ALIAS.get(val, val)))
        elif typ == "LOC":
            out.append((typ, LOC_ALIAS.get(val, val)))
        else:
            out.append((typ, val))
    return out

df["normalized_entities"] = df["normalized_entities"].apply(apply_aliases)

# --- Build per-professor property dictionaries based on sections ---

SECTION_KEYS = {
    "corporate": ["<h4>corporate experience</h4>", "corporate experience"],
    "academic_exp": ["<h4>academic experience</h4>", "academic experience"],
    "academic_bg": ["<h4>academic background</h4>", "academic background"],
}

DEGREE_PAT = re.compile(
    r"\b(ph\.?d|doctorate|msc|m\.sc\.|ms|ma|m\.a\.|mba|bsc|b\.sc\.|bs|ba|b\.a\.|llm|jd|md)\b",
    re.IGNORECASE
)
# light list of subjects to catch common terms (extend later if you want)
SUBJECT_PAT = re.compile(
    r"\b(machine learning|quantum physics|mathematics|finance|economics|marketing|law|computer science|data science)\b",
    re.IGNORECASE
)

def get_sections(raw_html_or_text:str):
    """Return dict with text per section; robust to having tags removed or present."""
    t = raw_html_or_text or ""
    t_l = t.lower()
    # find start indices
    idx = {}
    for key, variants in SECTION_KEYS.items():
        for v in variants:
            p = t_l.find(v)
            if p != -1:
                idx[key] = p
                break
    # slice text by nearest next section
    order = [k for k in ["corporate","academic_exp","academic_bg"] if k in idx]
    out = {"corporate":"", "academic_exp":"", "academic_bg":""}
    for i,k in enumerate(order):
        start = idx[k]
        end = idx[order[i+1]] if i+1 < len(order) else len(t)
        out[k] = re.sub(r"<.*?>"," ", t[start:end])  # strip tags inside slice
    # fallback: if nothing matched, put everything in academic_exp to not lose info
    if not any(out.values()):
        out["academic_exp"] = re.sub(r"<.*?>"," ", t)
    # compact whitespace
    for k in out:
        out[k] = re.sub(r"\s+"," ", out[k]).strip()
    return out

def build_property_dict(row):
    sections = get_sections(row.get("full_info","") or row.get("clean_text",""))
    ents = row["normalized_entities"]

    def pick_in(section_text, typ):
        hits = []
        S = section_text.lower()
        for t, val in ents:
            if t != typ: 
                continue
            v = val.lower()
            # string containment as a simple attribution heuristic
            if v and v in S and val not in hits:
                hits.append(val)
        return hits

    corp_orgs = pick_in(sections["corporate"], "ORG")
    corp_locs = pick_in(sections["corporate"], "LOC")
    acad_bg_orgs = pick_in(sections["academic_bg"], "ORG")
    acad_exp_orgs = pick_in(sections["academic_exp"], "ORG")
    acad_exp_locs = pick_in(sections["academic_exp"], "LOC")

    # Education (simple regex on academic background section)
    education = list({m.group(0).upper().replace(".", "") for m in DEGREE_PAT.finditer(sections["academic_bg"])})

    # Subjects (keywords from academic experience)
    subjects = list({m.group(0).title() for m in SUBJECT_PAT.finditer(sections["academic_exp"])})

    return {
        "Corporate Experience - Location": corp_locs,
        "Corporate Experience - Organization": corp_orgs,
        "Academic Background - Organization": acad_bg_orgs,
        "Academic Background - Education": education,
        "Academic Experience - Courses": [],  # optional—can be filled later if you choose
        "Academic Experience - Subjects": subjects,
    }

df["property_dict"] = df.apply(build_property_dict, axis=1)
df["property_dict"].head(2).to_dict()



from collections import Counter

org_counts = Counter([e[1] for row in df["normalized_entities"] for e in row if e[0] == "ORG"])
loc_counts = Counter([e[1] for row in df["normalized_entities"] for e in row if e[0] == "LOC"])

print("Top ORGs:", org_counts.most_common(10))
print("Top LOCs:", loc_counts.most_common(10))


In [ ]:
output_path = "../data/processed/teachers_db_cleaned.parquet"
df.to_parquet(output_path, index=False)
print(f"Saved cleaned dataset to {output_path}")

In [51]:
# ---- Build dictionaries per professor from sectioned entities ----
def to_lists(section_entities):
    orgs = [e[1] for e in section_entities if e[0] == "ORG"]
    locs = [e[1] for e in section_entities if e[0] == "LOC"]
    return orgs, locs

def build_professor_dict_row(row):
    corp_orgs, corp_locs = to_lists(row["norm_corp"])
    acadexp_orgs, acadexp_locs = to_lists(row["norm_acadexp"])
    acadbg_orgs, acadbg_locs = to_lists(row["norm_acadbg"])

    return {
        "Corporate Experience - Organization": corp_orgs,
        "Corporate Experience - Location": corp_locs,
        "Academic Background - Organization": acadbg_orgs,
        "Academic Background - Education": [],    # optional enhancement later (regex keywords)
        "Academic Experience - Courses": [],      # optional
        "Academic Experience - Subjects": [],     # optional
        # You can also include acadexp_locs if you want:
        # "Academic Experience - Location": acadexp_locs
    }

df["professor_dict"] = df.apply(build_professor_dict_row, axis=1)

# ---- Extract degrees/subjects from the "Academic Background" section and
# move them out of ORGs into Academic Background - Education ----

import re

def get_section(text, start_key, stop_keys):
    t = text or ""
    t_low = t.lower()
    s = t_low.find(start_key.lower())
    if s == -1:
        return ""
    e_candidates = [t_low.find(k.lower(), s+1) for k in stop_keys]
    e_candidates = [e for e in e_candidates if e != -1]
    e = min(e_candidates) if e_candidates else len(t)
    return t[s:e]

# Broad degree / credential patterns (English + common ES terms)
DEGREE_PAT = re.compile(
    r"""
    \b(
        ph\.?d\.?|doctor(?:ate)?\s+of\s+[A-Za-zÁÉÍÓÚÜÑ&\-\s]+|
        m\.?b\.?a\.?|m\.?sc\.?|m\.?s\.?|m\.?a\.?|ll\.?m\.?|
        b\.?sc\.?|b\.?s\.?|b\.?a\.?|
        master(?:'s)?\s+in\s+[A-Za-zÁÉÍÓÚÜÑ&\-\s]+|
        bachelor(?:'s)?\s+in\s+[A-Za-zÁÉÍÓÚÜÑ&\-\s]+|
        licenciatura\s+en\s+[A-Za-zÁÉÍÓÚÜÑ&\-\s]+|
        grado\s+en\s+[A-Za-zÁÉÍÓÚÜÑ&\-\s]+
    )\b
    """,
    re.IGNORECASE | re.VERBOSE,
)

# Light subject extractor: “… in X” or “… of X”
SUBJECT_PAT = re.compile(r"\b(?:in|of)\s+([A-Z][A-Za-zÁÉÍÓÚÜÑ&\-\s]{3,})")

def split_background_fields(row):
    # 1) Get the Academic Background text slice from your cleaned text
    bg_text = get_section(
        row.get("clean_text", ""),
        start_key="Academic Background",
        stop_keys=["Academic Experience", "Corporate Experience"]
    )

    # 2) Degrees & subjects from the text
    degrees = [m.group(0).strip().rstrip(",.;") for m in DEGREE_PAT.finditer(bg_text)]
    subjects = [m.group(1).strip().rstrip(",.;") for m in SUBJECT_PAT.finditer(bg_text)]

    # 3) Remove degree-like tokens that leaked into ORG buckets for this professor
    d = row["professor_dict"].copy()
    ab_orgs = d.get("Academic Background - Organization", [])
    cleaned_ab_orgs = []
    for org in ab_orgs:
        if DEGREE_PAT.search(org) or SUBJECT_PAT.search(org):
            continue
        cleaned_ab_orgs.append(org)

    # 4) Update dictionary
    d["Academic Background - Organization"] = cleaned_ab_orgs
    d["Academic Background - Education"] = sorted(set(d.get("Academic Background - Education", []) + degrees))
    # Optional: add subjects here (or keep for "Academic Experience - Subjects" later)
    d["Academic Experience - Subjects"] = sorted(set(d.get("Academic Experience - Subjects", []) + subjects))

    return d

df["professor_dict"] = df.apply(split_background_fields, axis=1)


import random
for i, d in enumerate(df["professor_dict"].sample(30, random_state=42).to_list(), 1):
    print(f"\n--- Professor {i} ---")
    for k, v in d.items():
        print(f"{k}: {v}")



--- Professor 1 ---
Corporate Experience - Organization: ['Instituto Cervantes', 'European Union Association Of National Cultural Institutes', 'Hispanic Observatory', 'Ronda Energy Ltd']
Corporate Experience - Location: ['Istanbul', 'Dublin', 'London', 'Turkey', 'Ireland', 'United Kingdom', 'United Kingdom', 'London']
Academic Background - Organization: ['Antony S College', 'University College', 'Rbonne University', 'Ortega Y Gasset University Institute', 'University College']
Academic Background - Education: ['B.A', 'M.A']
Academic Experience - Courses: []
Academic Experience - Subjects: ['History', 'International Relations']

--- Professor 2 ---
Corporate Experience - Organization: []
Corporate Experience - Location: []
Academic Background - Organization: []
Academic Background - Education: ['Bachelor in physics', 'Ph.D']
Academic Experience - Courses: []
Academic Experience - Subjects: []

--- Professor 3 ---
Corporate Experience - Organization: ['Environmental & Building Physics',